In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
checkpoint = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16)

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe

In [ ]:
message = [
    {
        "role": "user",
        "content": "你好"
    }
]
pipe(message, max_new_tokens=1000)[-1]["generated_text"][-1]["content"]

In [48]:
# 工具案例来自glm4官方 https://github.com/THUDM/GLM-4/blob/main/basic_demo/openai_api_request.py
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_gold_price",
            "description": "获取某天的金价",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "描述的日期，比方说 2025年1月"
                    }
                }
            },
            "required": ["date"]
        }
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "1999年的黄金价格如何？"
    }
]

input = tokenizer.apply_chat_template(messages, tools=tools, tokenize=False, add_generation_prompt=True)
input

In [ ]:
output = pipe(input, max_new_tokens=1000)[-1]["generated_text"]
tool_token = "<|im_start|>assistant"
index = output.rindex(tool_token)
answer = output[index+len(tool_token):]
answer

In [ ]:
messages = [
    {
        "role": "user",
        "content": "今天上海的天气怎么样"
    },
    {
        "role": "tool",
        "content": "27摄氏度"
    }
]
tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [ ]:
pipe(messages, max_new_tokens=1000)